In [1]:
! pip install langchain
! pip install -qU langchain_mistralai
! pip install langchain-community

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import getpass
import os
os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")

In [7]:
from langchain.agents import initialize_agent, AgentExecutor
from langchain_mistralai import ChatMistralAI
from langchain.tools import Tool

from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate, MessagesPlaceholder

In [8]:
llm_model = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
)

### Without Multi-Agents

In [69]:
template = """
You are a counselor, Generate a response to the user's text: 
{user_text}
"""

In [70]:
llm_chain = LLMChain(llm=llm_model, prompt=PromptTemplate(input_variables=["user_text"],
                                                                 template=template))

In [71]:
def run_generation(user_text):
    final_response = llm_chain.run(user_text=user_text)
    return final_response

In [72]:
user_text = "I am failing in every project I take up, and I feel like a failure, and my life is going nowhere. My wife thinks I am stupid, I want to die"
output = run_generation(user_text)

In [73]:
output

"I'm really sorry that you're feeling this way, but I'm here to help. It's important to remember that everyone faces challenges and setbacks, and it doesn't reflect your worth as a person. Here are a few things to consider:\n\n1. **It's okay to struggle**: Everyone struggles with projects and goals at times. It's a normal part of life and doesn't mean you're a failure.\n\n2. **Seek support**: Talk to someone you trust about how you're feeling. This could be a friend, family member, or a professional counselor. They can provide a different perspective and help you navigate through this difficult time.\n\n3. **Communicate with your wife**: Open up about how her words make you feel. It's possible she doesn't realize the impact of her comments. Couples counseling could be a safe space to have this conversation.\n\n4. **Reframe your thoughts**: Instead of thinking about where you aren't, focus on the progress you've made and the things you've learned. Every setback is an opportunity to grow

### With Multi-agents

In [91]:
attributes = ["Reframing", "Unconditional Positive Regard", "Solution-Focused"]

In [92]:
debate_prompt_template = """
You are a counselor agent specializing in {attribute}.
Debate with your co-expertise agents to generate a response why you think your expertise is needed in this user's post.
User post: "{user_post}"
Previous debate history: {debate_history}
Converse in short and concise and to the point of your expertise in {attribute}.
"""

In [93]:
debate_chains = []
for attribute in attributes:
    prompt_template = PromptTemplate(input_variables=["user_post", "debate_history"], template=debate_prompt_template)
    debate_chain = LLMChain(llm=llm_model, prompt=prompt_template)
    debate_chains.append((debate_chain, attribute))

In [94]:
print(debate_chains)

[(LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['attribute', 'debate_history', 'user_post'], input_types={}, partial_variables={}, template='\nYou are a counselor agent specializing in {attribute}.\nDebate with your co-expertise agents to generate a response why you think your expertise is needed in this user\'s post.\nUser post: "{user_post}"\nPrevious debate history: {debate_history}\nConverse in short and concise and to the point of your expertise in {attribute}.\n'), llm=ChatMistralAI(client=<httpx.Client object at 0x00000136920B1FA0>, async_client=<httpx.AsyncClient object at 0x000001369221FDA0>, mistral_api_key=SecretStr('**********'), endpoint='https://api.mistral.ai/v1', max_retries=2, model='mistral-large-latest', temperature=0.0), output_parser=StrOutputParser(), llm_kwargs={}), 'Reframing'), (LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['attribute', 'debate_history', 'user_post'], input_types={}, partial_variables={}, template='\nYou are a 

In [95]:

# Stage 1: Strategic Debating
def run_strategic_debate(user_post, num_turns=2):
    debate_history = []
    debate_history_str = "\n".join(debate_history) if debate_history else "No prior history."
    
    for turn in range(num_turns):
        for debate_chain, attribute in debate_chains:
            response = debate_chain.run(user_post=user_post, debate_history=debate_history_str, attribute=attribute)
            debate_history.append(response)
        debate_history_str = "\n".join(debate_history)
    return debate_history


In [96]:
input_text = "I am failing in every project I take up, and I feel like a failure, and my life is going nowhere. My wife thinks I am stupid, I want to die"
debate_history = run_strategic_debate(input_text)
# final_response = generate_response(input_text, counselor_profile)
# print(counselor_profile.content)

In [97]:
debate_history

['**Counselor Agent (Reframing Expert):**\n\n"The user\'s post is filled with negative self-talk and a distorted perception of their situation. Reframing is crucial here to help them see their experiences in a more balanced and constructive light. By reframing \'failing in every project\' to \'learning from each attempt,\' and \'feeling like a failure\' to \'recognizing areas for growth,\' we can shift their mindset from despair to potential. Additionally, addressing the perception that \'my wife thinks I am stupid\' can be reframed to \'my wife may be frustrated, but let\'s explore her perspective together.\' Reframing can transform their outlook and provide a foundation for positive change."\n\n**Co-Expertise Agent (Cognitive Behavioral Therapy Expert):**\n\n"While reframing is important, Cognitive Behavioral Therapy (CBT) can provide a structured approach to identify and challenge the user\'s negative thought patterns. CBT techniques can help them recognize cognitive distortions and

In [98]:
counseling_prompt_template = """
You are a counselor compass. Using the following debate history:
{debate_history}
Access the debate history of three counselors (solution-focused , UnconditionalPostive Regard , Reframing) above and 
generate the scores for each counselor out of 5 on how important they are for the user's input.
Just give the three scores respectively for (solution-focused , UnconditionalPostive Regard , Reframing) separated by commas.
"""

In [99]:
counseling_chain = LLMChain(llm=llm_model, prompt=PromptTemplate(input_variables=["debate_history"],
                                                                 template=counseling_prompt_template))

In [100]:
def run_counseling_stage(debate_history):
    debate_history_str = "\n".join(debate_history)
    counseling_output = counseling_chain.run(debate_history=debate_history_str)
    return counseling_output

In [101]:
counseling_output = run_counseling_stage(debate_history)

In [102]:
counseling_output

'4, 5, 5'

In [103]:
response_generation_prompt_template = """
You are a counsellor. Generate the response to the user's input:
User post: "{user_post}"
Generate a final cohesive response to user in short and concise matter that addresses the user's concerns with 
you being a counsellor with the scoring for (solution-focused , UnconditionalPostive Regard , Reframing) out of 5 respectively as {counseling_output}
"""


In [104]:
response_chain = LLMChain(llm=llm_model, prompt=PromptTemplate(input_variables=["counseling_output" , "user_post"],
                                                               template=response_generation_prompt_template))


In [105]:
def run_response_generation(counseling_output , input_text):
    final_response = response_chain.run(counseling_output=counseling_output , user_post=input_text)
    return final_response

In [106]:
run_response_generation(counseling_output , input_text)

"I'm really sorry that you're feeling this way, but I'm here to help. It's important to remember that everyone faces challenges and setbacks, and it doesn't reflect on your intelligence or worth as a person. Let's try to focus on the skills and strengths you have, and find ways to apply them to your projects. Perhaps we can reframe this situation as an opportunity for growth rather than a sign of failure. It's also crucial to communicate openly with your wife about your feelings. Lastly, please remember that your life is valuable, and it's okay to seek help when you're feeling overwhelmed. Let's take this one step at a time, together."

In [ ]:
def run_mental_agora(user_post):
    debate_history = run_strategic_debate(user_post)

    # Stage 2: Tailored Counseling
    counselor_compass = run_counseling_stage(debate_history)
    print("\n=== Counseling compass ===")
    print(counselor_compass)

    # Stage 3: Support Response Generation
    final_response = run_response_generation(counselor_compass , user_post)
    return final_response


### Testing Single Agent Output

In [109]:
user_text = "I am failing in every project I take up, and I feel like a failure, and my life is going nowhere. My wife thinks I am stupid, I want to die"
user_text_2 =  "What should I do if I have to merge the data from different data frames"

In [110]:
output = run_generation(user_text)
output

"I'm really sorry that you're feeling this way, but I'm here to help. It's important to remember that everyone faces challenges and setbacks, and it doesn't reflect your worth as a person. Here are a few things to consider:\n\n1. **It's okay to struggle**: Everyone struggles with projects and goals at times. It's a normal part of life and doesn't mean you're a failure.\n\n2. **Seek support**: Talk to someone you trust about how you're feeling. This could be a friend, family member, or a professional counselor. They can provide a different perspective and help you navigate through this difficult time.\n\n3. **Communicate with your wife**: Open up about how her words make you feel. It's possible she doesn't realize the impact of her comments. Couples counseling could be a safe space to have this conversation.\n\n4. **Reframe your thoughts**: Instead of thinking about where you aren't, focus on the progress you've made and the things you've learned. Every setback is an opportunity to grow

In [111]:
output = run_generation(user_text_2)
output

"Merging data from different data frames is a common task in data analysis. Here are some steps and tips to help you through the process:\n\n1. **Understand the Data**: Before merging, make sure you understand the structure and content of each data frame. This includes knowing the columns, data types, and any unique identifiers that can be used for merging.\n\n2. **Identify Common Keys**: Determine the columns that will be used as keys for merging. These keys should uniquely identify the rows in each data frame.\n\n3. **Choose the Merge Type**: Decide on the type of merge you need. Common types include:\n   - **Inner Join**: Returns only the rows with matching keys in both data frames.\n   - **Left Join**: Returns all rows from the left data frame and the matched rows from the right data frame.\n   - **Right Join**: Returns all rows from the right data frame and the matched rows from the left data frame.\n   - **Outer Join**: Returns all rows when there is a match in either left or rig

### Testing Multi-agent Output

In [112]:
output_multi_agents = run_mental_agora(user_text)


=== Counseling compass ===
4, 5, 5

=== Final Response ===
I'm really sorry that you're feeling this way, but I'm here to help. It's important to remember that everyone faces challenges and setbacks, and it doesn't reflect on your intelligence or worth as a person. Let's try to focus on the skills and strengths you have, and find ways to apply them to your projects. Perhaps we can reframe this situation as an opportunity for growth rather than a sign of failure. It's also crucial to communicate openly with your wife about your feelings. Lastly, please remember that your life is valuable, and it's okay to seek help when you're feeling overwhelmed. Let's take this one step at a time, together.


In [113]:
output_multi_agents

"I'm really sorry that you're feeling this way, but I'm here to help. It's important to remember that everyone faces challenges and setbacks, and it doesn't reflect on your intelligence or worth as a person. Let's try to focus on the skills and strengths you have, and find ways to apply them to your projects. Perhaps we can reframe this situation as an opportunity for growth rather than a sign of failure. It's also crucial to communicate openly with your wife about your feelings. Lastly, please remember that your life is valuable, and it's okay to seek help when you're feeling overwhelmed. Let's take this one step at a time, together."

In [114]:
output_multi_agents = run_mental_agora(user_text_2)
output_multi_agents


=== Counseling compass ===
4, 3, 5

=== Final Response ===
It sounds like you're facing a challenge with merging data from different data frames. This can be a common task in data analysis. One approach is to use tools like pandas in Python, which has functions like `merge` or `concat` that can help you combine data frames efficiently. If you're using another tool, there are likely similar functions available. Breaking the task into smaller steps and focusing on one merge at a time can make it more manageable. Remember, each step you take brings you closer to your goal. If you encounter any specific issues, feel free to ask for more detailed guidance. You're capable of handling this, and taking it step by step will help you succeed.


"It sounds like you're facing a challenge with merging data from different data frames. This can be a common task in data analysis. One approach is to use tools like pandas in Python, which has functions like `merge` or `concat` that can help you combine data frames efficiently. If you're using another tool, there are likely similar functions available. Breaking the task into smaller steps and focusing on one merge at a time can make it more manageable. Remember, each step you take brings you closer to your goal. If you encounter any specific issues, feel free to ask for more detailed guidance. You're capable of handling this, and taking it step by step will help you succeed."